# Entity Matching (EM) about Books

# Introduction

This IPython notebook shows a basic workflow two tables using *py_entitymatching*. We want to match data science books in library of UW-Madison and UIUC.  The book information of UW-Madison is from [here](https://search.library.wisc.edu/search/system?q=Data+Science) and the book information of UIUC is from [here](https://vufind.carli.illinois.edu/vf-uiu/Search/Home?lookfor=Data+Science+&type=all&start_over=1&submit=Find&search=new). Details can be found from our Stage 2 Report [here](https://github.com/iphyer/CS839ClassProject/blob/master/stage2/Stage2Report.pdf). 


First, we need to import *py_entitymatching* package and other libraries as follows:

In [7]:
import pandas as pd
import py_entitymatching as em

# Read input tables

We begin by loading the input tables.

We name the table about UW-Madison `TableA.csv` and the table about UIUC `TableB.csv`. And there are 

* 4824 tuples in table `TableA.csv`
* 5060 tuples in table `TableB.csv`

In [8]:
table_A = em.read_csv_metadata('../data/TableA.csv', key = 'ID')
table_B = em.read_csv_metadata('../data/TableB.csv', key = 'ID')

Metadata file is not present in the given path; proceeding to read the csv file.
Metadata file is not present in the given path; proceeding to read the csv file.


In [9]:
table_A.shape

(4824, 8)

In [10]:
table_B.shape

(5060, 8)

# Down sampling
Down sampling table A and B， get 1000 examples from both table A and B.

In [11]:
A, B = em.down_sample(table_A, table_B, size=1000, y_param = 1, show_progress=False)

In [12]:
A.shape

(1000, 8)

In [13]:
block_f = em.get_features_for_blocking(A, B)
block_t = em.get_tokenizers_for_blocking()
block_s = em.get_sim_funs_for_blocking()
r = em.get_feature_fn('jaccard(dlm_dc0(ltuple["Title"]), dlm_dc0(rtuple["Title"]))', block_t, block_s)
em.add_feature(block_f, 'Title_Title_jac_dlm_dc0_dlm_dc0', r)

The table shows the corresponding attributes along with their respective types.
Please confirm that the information  has been correctly inferred.
If you would like to skip this validation process in the future,
please set the flag validate_inferred_attr_types equal to false.


,Left Attribute,Right Attribute,Left Attribute Type,Right Attribute Type,Example Features
0,ID,ID,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
1,Title,Title,short string (1 word),short string (1 word),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
2,Author,Author,medium string (5 words to 10 words),short string (1 word to 5 words),Not Applicable: Types do not match
3,Publication,Publication,medium string (5 words to 10 words),short string (1 word to 5 words),Not Applicable: Types do not match
4,Format,Format,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
5,ISBN,ISBN,numeric,numeric,Exact Match; Absolute Norm
6,Series,Series,medium string (5 words to 10 words),medium string (5 words to 10 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
7,Physical Details,Physical Details,short string (1 word),short string (1 word to 5 words),Not Applicable: Types do not match


Do you want to proceed? (y/n):y


True

# Block tables to get candidate set

Here we will use several blockers to remove obviously non-matching tuple pairs from the input tables.

For the same book, since we got the data from two different library websites, their attributes may not be the exact same. Therefore, we applied an OverlapBlocker over some of the attributes, including the *Title* and *Author*.

After multiple tests, we found the best overlap_size for each attribute - for *Author* and *Title*, we set the overlap_size to be 2 and 4 respectively.

In [14]:
ob = em.OverlapBlocker()
C = ob.block_tables(A, B, 'Author', 'Author', 
                    l_output_attrs=['Title','Author','Publication','Format','ISBN','Series', 'Physical Details'], 
                    r_output_attrs=['Title','Author','Publication','Format','ISBN','Series', 'Physical Details'], 
                    overlap_size = 2)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [15]:
D = ob.block_candset(C, 'Title', 'Title', overlap_size = 4)

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [47]:
len(D)
D.to_csv('Set_C.csv', sep = ',')

## Sampling from D
Sample 300 examples from D.

In [ ]:
S = em.sample_table(D, 300)

## Create label
After manually labeling the data, We get 300 candidates with labels in label_S. <br/>
Also, need to set the metadata for label_S appropriately.

In [3]:
label_S = pd.read_csv('./Set_G.csv')
# em.copy_properties(S, label_S)
em.set_property(label_S, 'key', '_id')
em.set_property(label_S, 'fk_ltable', 'ltable_ID')
em.set_property(label_S, 'fk_rtable', 'rtable_ID')
label_S_rtable = em.read_csv_metadata('./label_S_rtable.csv')
label_S_ltable = em.read_csv_metadata('./label_S_ltable.csv')
em.set_property(label_S, 'rtable', label_S_rtable)
em.set_property(label_S, 'ltable', label_S_ltable)

True

In [4]:
IJ = em.split_train_test(label_S, train_proportion=0.66, random_state=0)
I = IJ['train']
J = IJ['test']

In [45]:
I.to_csv('Set_I.csv', sep = ',')

In [46]:
J.to_csv('Set_J.csv', sep = ',')

# Training

In [86]:
match_f = em.get_features_for_matching(A, B)

The table shows the corresponding attributes along with their respective types.
Please confirm that the information  has been correctly inferred.
If you would like to skip this validation process in the future,
please set the flag validate_inferred_attr_types equal to false.


,Left Attribute,Right Attribute,Left Attribute Type,Right Attribute Type,Example Features
0,ID,ID,short string (1 word),short string (1 word),Levenshtein Distance; Levenshtein Similarity
1,Title,Title,short string (1 word),short string (1 word),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
2,Author,Author,medium string (5 words to 10 words),short string (1 word to 5 words),Not Applicable: Types do not match
3,Publication,Publication,medium string (5 words to 10 words),short string (1 word to 5 words),Not Applicable: Types do not match
4,Format,Format,short string (1 word to 5 words),short string (1 word to 5 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
5,ISBN,ISBN,numeric,numeric,Exact Match; Absolute Norm
6,Series,Series,medium string (5 words to 10 words),medium string (5 words to 10 words),"Jaccard Similarity [3-grams, 3-grams]; Cosine Similarity [Space Delimiter, Space Delimiter]"
7,Physical Details,Physical Details,short string (1 word),short string (1 word to 5 words),Not Applicable: Types do not match


Do you want to proceed? (y/n):y


# Generating feature
In the feature engineering process, we first exclude the features that are automatically generated by ID and ISBN, and add Jaccard scores on Book Title, Author, Publication and Series with space as the delimiter.  We fit 19 features into Decision Tree, Random Forest, SVM, Naive Bayes, Logistic Regression, Linear Regression with 5-fold cross-validation.

In [87]:
match_t = em.get_tokenizers_for_matching()
match_s = em.get_sim_funs_for_matching()
f1 = em.get_feature_fn('jaccard(dlm_dc0(ltuple["Title"]), dlm_dc0(rtuple["Title"]))', match_t, match_s)
f2 = em.get_feature_fn('jaccard(dlm_dc0(ltuple["Author"]), dlm_dc0(rtuple["Author"]))', match_t, match_s)
f3 = em.get_feature_fn('jaccard(dlm_dc0(ltuple["Publication"]), dlm_dc0(rtuple["Publication"]))', match_t, match_s)
f4 = em.get_feature_fn('jaccard(dlm_dc0(ltuple["Series"]), dlm_dc0(rtuple["Series"]))', match_t, match_s)
em.add_feature(match_f, 'Title_Title_jac_dlm_dc0_dlm_dc0', f1)
em.add_feature(match_f, 'Author_Author_jac_dlm_dc0_dlm_dc0', f2)
em.add_feature(match_f, 'Publication_Publication_jac_dlm_dc0_dlm_dc0', f3)
em.add_feature(match_f, 'Series_Series_jac_dlm_dc0_dlm_dc0', f4)

True

# debugging
After first round of debug, we found that the learning method had difficulty in distinguishing books/journals in different versions/edition/conference. So we introduced a new feature that capture this piece of information embedded in book titles. Basically, we extracted the roman numerals from the title and use 1-0 to indicate whether they are the same from each tuple pair.

In [88]:
# Add blackbox feature

import re
# for Roman numerals matching
def Title_Title_blackbox_1(x, y):
    
    # get name attribute
    x_title = x['Title']
    y_title = y['Title']
    regex_roman = '\s+[MDCLXVI]+\s+'
    x_match = None
    y_match = None
    if re.search(regex_roman, x_title):
        x_match = re.search(regex_roman, x_title).group(0)
    if re.search(regex_roman, y_title):
        y_match = re.search(regex_roman, y_title).group(0)

    if x_match is None or y_match is None:
        return False
    else:
        return x_match == y_match

em.add_blackbox_feature(match_f, 'blackbox_1', Title_Title_blackbox_1)

True

Here we delete features that are related to ID and ISBN.

In [89]:
match_f = match_f[(match_f['left_attribute'] != 'ID') & (match_f['left_attribute'] != 'ISBN')]

# Training

Extract feature from set I.

In [90]:
H = em.extract_feature_vecs(I, feature_table=match_f, attrs_after=['label'])

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [91]:
dt = em.DTMatcher(name='DecisionTree', random_state = 0, max_depth = 5)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name = 'NaiveBayes')

result = em.select_matcher(matchers=[dt, rf, svm, lg, ln, nb], 
                           table=H, 
                           exclude_attrs=['_id', 'ltable_ID', 'rtable_ID'], 
                           target_attr='label', 
                           k=5,
                           metric_to_select_matcher='precision'
                           )

In [85]:
result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.886450,0.882262,0.884683
1,RF,0.921318,0.895359,0.880092
2,SVM,0.829167,0.664124,0.731630
3,LogReg,0.870488,0.822088,0.828911
4,LinReg,0.909921,0.925595,0.920403
5,NaiveBayes,0.779401,0.869124,0.807932


# Predicting

In [92]:
# RF

rf.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
       target_attr='label')

H_test = em.extract_feature_vecs(J, feature_table=match_f, attrs_after=['label'])
pred_table = rf.predict(table= H_test, 
                        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
                        target_attr='predicted_labels', 
                        return_probs=True, 
                        probs_attr='proba', 
                        append=True)
eval_summary = em.eval_matches(pred_table, 'label', 'predicted_labels')
eval_summary

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


OrderedDict([('prec_numerator', 30.0),
             ('prec_denominator', 32.0),
             ('precision', 0.9375),
             ('recall_numerator', 30.0),
             ('recall_denominator', 31.0),
             ('recall', 0.967741935483871),
             ('f1', 0.9523809523809523),
             ('pred_pos_num', 32.0),
             ('false_pos_num', 2.0),
             ('false_pos_ls', [('a5426', 'b3056'), ('a3061', 'b99')]),
             ('pred_neg_num', 70.0),
             ('false_neg_num', 1.0),
             ('false_neg_ls', [('a3595', 'b826')])])

In [93]:
# DT

dt.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
       target_attr='label')

H_test = em.extract_feature_vecs(J, feature_table=match_f, attrs_after=['label'])
pred_table = dt.predict(table= H_test, 
                        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
                        target_attr='predicted_labels', 
                        return_probs=True, 
                        probs_attr='proba', 
                        append=True)
eval_summary = em.eval_matches(pred_table, 'label', 'predicted_labels')
eval_summary

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


OrderedDict([('prec_numerator', 25.0),
             ('prec_denominator', 27.0),
             ('precision', 0.9259259259259259),
             ('recall_numerator', 25.0),
             ('recall_denominator', 31.0),
             ('recall', 0.8064516129032258),
             ('f1', 0.8620689655172414),
             ('pred_pos_num', 27.0),
             ('false_pos_num', 2.0),
             ('false_pos_ls', [('a5146', 'b695'), ('a3709', 'b907')]),
             ('pred_neg_num', 75.0),
             ('false_neg_num', 6.0),
             ('false_neg_ls',
              [('a4779', 'b2500'),
               ('a2488', 'b767'),
               ('a1614', 'b3909'),
               ('a3595', 'b826'),
               ('a2082', 'b2692'),
               ('a5196', 'b5855')])])

In [41]:
# SVM

svm.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
       target_attr='label')

H_test = em.extract_feature_vecs(J, feature_table=match_f, attrs_after=['label'])
pred_table = svm.predict(table= H_test, 
                        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
                        target_attr='predicted_labels', 
                        return_probs=False, 
                        probs_attr='proba', 
                        append=True)
eval_summary = em.eval_matches(pred_table, 'label', 'predicted_labels')
eval_summary

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


OrderedDict([('prec_numerator', 21.0),
             ('prec_denominator', 31.0),
             ('precision', 0.6774193548387096),
             ('recall_numerator', 21.0),
             ('recall_denominator', 31.0),
             ('recall', 0.6774193548387096),
             ('f1', 0.6774193548387096),
             ('pred_pos_num', 31.0),
             ('false_pos_num', 10.0),
             ('false_pos_ls',
              [('a4814', 'b140'),
               ('a5426', 'b3056'),
               ('a3061', 'b99'),
               ('a5426', 'b592'),
               ('a5640', 'b685'),
               ('a188', 'b4616'),
               ('a5426', 'b695'),
               ('a2905', 'b1241'),
               ('a4984', 'b1911'),
               ('a5426', 'b685')]),
             ('pred_neg_num', 71.0),
             ('false_neg_num', 10.0),
             ('false_neg_ls',
              [('a420', 'b5252'),
               ('a4451', 'b928'),
               ('a3594', 'b858'),
               ('a120', 'b4255'),
            

In [42]:
# LR

lg.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
       target_attr='label')

H_test = em.extract_feature_vecs(J, feature_table=match_f, attrs_after=['label'])
pred_table = lg.predict(table= H_test, 
                        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
                        target_attr='predicted_labels', 
                        return_probs=False, 
                        probs_attr='proba', 
                        append=True)
eval_summary = em.eval_matches(pred_table, 'label', 'predicted_labels')
eval_summary

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


OrderedDict([('prec_numerator', 27.0),
             ('prec_denominator', 32.0),
             ('precision', 0.84375),
             ('recall_numerator', 27.0),
             ('recall_denominator', 31.0),
             ('recall', 0.8709677419354839),
             ('f1', 0.8571428571428571),
             ('pred_pos_num', 32.0),
             ('false_pos_num', 5.0),
             ('false_pos_ls',
              [('a3709', 'b907'),
               ('a4969', 'b907'),
               ('a5146', 'b695'),
               ('a5426', 'b3056'),
               ('a625', 'b685')]),
             ('pred_neg_num', 70.0),
             ('false_neg_num', 4.0),
             ('false_neg_ls',
              [('a4779', 'b2500'),
               ('a3595', 'b826'),
               ('a1876', 'b862'),
               ('a3020', 'b4720')])])

Extract feature from set J.

In [43]:
# LN

ln.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
       target_attr='label')

H_test = em.extract_feature_vecs(J, feature_table=match_f, attrs_after=['label'])
pred_table = ln.predict(table= H_test, 
                        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
                        target_attr='predicted_labels', 
                        return_probs=False, 
                        probs_attr='proba', 
                        append=True)
eval_summary = em.eval_matches(pred_table, 'label', 'predicted_labels')
eval_summary

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


OrderedDict([('prec_numerator', 28.0),
             ('prec_denominator', 31.0),
             ('precision', 0.9032258064516129),
             ('recall_numerator', 28.0),
             ('recall_denominator', 31.0),
             ('recall', 0.9032258064516129),
             ('f1', 0.9032258064516129),
             ('pred_pos_num', 31.0),
             ('false_pos_num', 3.0),
             ('false_pos_ls',
              [('a5146', 'b695'), ('a2493', 'b928'), ('a2714', 'b2276')]),
             ('pred_neg_num', 71.0),
             ('false_neg_num', 3.0),
             ('false_neg_ls',
              [('a4779', 'b2500'), ('a3595', 'b826'), ('a1876', 'b862')])])

In [79]:
# NB

nb.fit(table=H, 
       exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
       target_attr='label')

H_test = em.extract_feature_vecs(J, feature_table=match_f, attrs_after=['label'])
pred_table = nb.predict(table= H_test, 
                        exclude_attrs=['_id', 'ltable_ID', 'rtable_ID', 'label'], 
                        target_attr='predicted_labels', 
                        return_probs=False, 
                        probs_attr='proba', 
                        append=True)
eval_summary = em.eval_matches(pred_table, 'label', 'predicted_labels')
eval_summary

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


OrderedDict([('prec_numerator', 27.0),
             ('prec_denominator', 40.0),
             ('precision', 0.675),
             ('recall_numerator', 27.0),
             ('recall_denominator', 31.0),
             ('recall', 0.8709677419354839),
             ('f1', 0.7605633802816901),
             ('pred_pos_num', 40.0),
             ('false_pos_num', 13.0),
             ('false_pos_ls',
              [('a5399', 'b685'),
               ('a4324', 'b2244'),
               ('a3709', 'b907'),
               ('a4969', 'b907'),
               ('a5146', 'b695'),
               ('a5426', 'b3056'),
               ('a2493', 'b928'),
               ('a3061', 'b99'),
               ('a5640', 'b685'),
               ('a5640', 'b907'),
               ('a625', 'b685'),
               ('a105', 'b2244'),
               ('a1638', 'b685')]),
             ('pred_neg_num', 62.0),
             ('false_neg_num', 4.0),
             ('false_neg_ls',
              [('a4779', 'b2500'),
               ('a420', 'b5